 <div style="border:1px solid #000; border-radius:1px; background:#fff;">
    <div style="padding:10px 10px 10px 0px; margin:3px; border-radius:1px; background:#178383; text-align:center;">
        <span style="font-family:sans-serif; font-size:40px; color:#fff;">Basics of Linear mixed models with R programming</span>
    </div>
</div>


* [dataset](https://data.mendeley.com/datasets/69p62ksdh6/6) 
* [reference](https://journals.sagepub.com/doi/full/10.1177/2515245920960351)

# Linear mixed models (LMMs):

These are statistical models that incorporate both fixed and random effects to accurately represent non-independent data structures. LMMs are an extension of simple linear models to allow both fixed and random effects, and are particularly used when there is non-independence in the data, such as arises from a hierarchical structure. They are often preferred over traditional analysis of variance regression models because they can deal with messy data and allow the use of all data, even when there are low sample sizes, structured data, and many covariates to fit. LMMs are applied in many disciplines where multiple correlated measurements are made on each subject, and they are prominently used in research involving human and animal subjects in fields ranging from psychology to baseball analytics and industrial statistics.

## Load Packages

First we have to install require packages if it not previously installed. Then we can call the packages by using `library()` method


In [ ]:
install.packages('ggplot2')
install.packages("lme4")
#install.packages("afex")

For Some reason `afex` did not installed here. It provides p value in our model summary. That is why you wont see p vlues in output but found in my writing as I ran it in my local machine also.

In [ ]:
library(ggplot2)
library(lme4)
#library(afex)

## Read data

In [ ]:
# read excel data in r

library(readxl)
df<-read_excel("/kaggle/input/psycological-data/Human Conciousness - Raw data.xlsx")


In [ ]:
head(df)


In [ ]:
#  only use id, sex, age, year, faculty , emotion to create new data frame

data <- df[,c("Sex","Age","Year","Faculty","Imagination","Emotion")]
head(data)


The dataset that we created contain six variables which includes gender, imagination emotions.
The variables we have:

`Sex`: Gender of respondents

`Age`: Age of the participant (presumably in years).

`Year`: Academic year or level of the participant

`Faculty`: Academic faculty or department to which the participant belongs

`Imagination`:  A variable representing some measure of imagination.

`Emotion`: A variable representing some measure of emotion



We want to investigate how the emotion score differs in participants between faculties and which influences emotion scores.

In [ ]:
summary(data)

## Prepare data

We have to convert our some variables in factors which are now in numeric format.


In [ ]:
# convert faculty, sex , year in factor

data$Faculty <- factor(data$Faculty)
data$Sex <- factor(data$Sex)
data$Year <- factor(data$Year)


In [ ]:
#  recode faculty level with  eadh number such as 1 become Faculty1

data$Faculty <- as.factor(data$Faculty)
levels(data$Faculty) <- paste0("Faculty", levels(data$Faculty))


In [ ]:
table(data$Faculty)

In [ ]:
str(data)

Now we can see that our desired variables is in factor format. Now we will try different model and compare between them.
First we used a simple linear regression model.

## linear regression

In [ ]:
# run a simple regression wherer emotion is dependent and sex age year faculty imagination is independent

model0 <- lm(Emotion ~ Sex + Age + Year + Faculty + Imagination, data = data)
summary(model0)

In [ ]:
plot(model0,3)

In this linear regression model, the Emotion score is significantly influenced by several factors. Participants with higher Imagination scores tend to have higher Emotion scores. Additionally, being in certain faculties, such as Faculty 5, is associated with significantly lower Emotion scores. Sex and Age also show significant associations with Emotion. The overall model has a statistically significant fit, explaining 33.43% of the variance in Emotion scores.

# Linear mixed models

There are several approaches when using multilevel analysis. We are
presenting here  for three cases:
 * Intercept-only Model,
 * random intercept and
 * random slope model.



Here we are visuaizing relation of Emotion and Imagination across different Faculties.

In [ ]:

# emotions by imagination across faculties
options(repr.plot.width=15, repr.plot.height=8)
ggplot(data = data, aes(x = Imagination, y = Emotion, color = Faculty)) +
  geom_point() +
  facet_wrap(~ Faculty)+ theme(plot.title = element_text(size = 26, hjust = 0.5))+
  labs(title = "Emotion by Imagination across Faculties",
       x = "Imagination",
       y = "Emotion")


### Intercept only model

In the context of linear mixed effects models, an "intercept-only model" is a model that includes only the intercept term without any fixed or random slopes. This type of model is appropriate when there is no meaningful variation in the slopes across different groups or levels of the data. It is commonly used as a baseline model for comparison with more complex models.

In [ ]:
#  intercept only model for emotion as dependent and faculty cluster variable

model1 <- lmer(Emotion ~ 1 + (1 | Faculty), data = data)
summary(model1)


In [ ]:
plot(model1)

In this linear mixed-effects model, the intercept represents the overall Emotion score across all faculties. The estimated fixed effect for the intercept is 4.02023 (p < 0.001), indicating the average Emotion score when considering the random effect of Faculty. The random effects show that there is variability in Emotion scores across different faculties, with a standard deviation of 0.1204, suggesting that the impact of Faculty on Emotion scores varies among groups.

## Random Intercept with One Fixed Level-1 Factor (Non-Random Slope)

A random intercept model is a type of linear mixed model that includes a random intercept term to account for the correlation among observations within the same group or cluster. The random intercept allows the average outcome to vary across different groups, capturing the unobserved group-specific effects. This model is suitable for data with a hierarchical or clustered structure, where observations are nested within higher-level groups (e.g., students within schools). The random intercept model consists of a fixed part (including the intercept and possibly other fixed effects) and a random part (the random intercept). The random intercept is treated as a random variable, and its variance is estimated from the data. This model is used in various fields, including education, psychology, and public health, to analyze data with a nested structure and account for group-level variability.

The basic syntax for mixed-effects modeling for an experiment with one independent variable (Imagination) and random intercepts but no random slopes for  items(faculty) is:

In [ ]:
# create a random intercept model

model2 <- lmer(Emotion ~ 1 + Imagination + (1 | Faculty), data = data)
summary(model2)


The portions in the interior sets of parentheses are the random effects, and the portions not in these parentheses are the fixed effects.The vertical lines within the random-effects portions of the code are called pipes, and they indicate that within each set of parentheses, the effects to the left of the pipe vary by the grouping factor to the right of the pipe.Thus, in this example, the intercept (indicated by the 1) varies by the one grouping factors in this experiment: Faculty. The model thus far includes random intercepts but no random slopes.

In [ ]:
plot(model2)

In this linear mixed-effects model, the Emotion scores are significantly influenced by Imagination. Participants with higher Imagination scores tend to have higher Emotion scores (estimate = 0.529, p < 0.001). The random effects indicate that there is variability in Emotion scores across different faculties, with a standard deviation of 0.0774. The high negative correlation (-0.960) between the Intercept and Imagination suggests that faculties with higher average Emotion scores tend to have a smaller positive effect of Imagination. Overall, the model with Imagination as the sole predictor provides a good fit to the data, explaining the variability in Emotion scores.

## Comparison between model1 and moedl2

In [ ]:
#  compare model 1 and model2

anova(model2,model1, refit = FALSE)


The model2 fits the data better - this is shown by the significant $\chi^2$ value. The test checks the difference between the two models in their deviance, i.e. the misfit of the models to the data.

This is evident from the considerably lower AIC (1220.016) and BIC (1238.755) values for Model 2. The likelihood ratio test (Chisq) further supports the superiority of Model 2 (p < 0.001), indicating that it provides a more parsimonious and statistically significant representation of the observed data.



## Random Intercept and Slope for One Level-1 Factor
A random slope model is a type of linear mixed model that includes a random slope term in addition to the random intercept term. In a random slope model, the relationship between the explanatory variable and the response variable can vary across different groups, allowing for individual differences in the effect of the predictor. This model is suitable for data with a hierarchical or clustered structure, where observations are nested within higher-level groups (e.g., students within schools).

In [ ]:
# create random slope model

model3 <- lmer(Emotion ~ 1 + Imagination + (1 + Imagination | Faculty), data = data)
summary(model3)


Here, the portions in parentheses indicate that both the intercept (indicated by the 1, which in this case is optional because it is implied by the presence of random slopes but is included for clarity) and the predictor(Imagination) (indicated by + Imagination) vary by faculties.
The model above includes only one predictor, but if a model includes multiple predictors the researcher may decide which of the predictors can vary by faculties; in other words, any fixed effect to the left of the interior parentheses can be included to the left of the pipe (inside the interior parentheses), provided that including it is justified given the design of the experiment.

In [ ]:
plot(model3)

In this linear mixed-effects model, Emotion scores are significantly influenced by both the intercept and Imagination. The random effects for Faculty account for variability in both intercepts and Imagination slopes across different faculties. The high negative correlation (-0.991) between the intercept and Imagination suggests that faculties with higher average Emotion scores tend to have a smaller positive effect of Imagination. The model provides a good fit to the data, with the intercept representing the baseline Emotion score and Imagination contributing positively.

## Comparison between model2 and modele3

In [ ]:
# compare model2 and model3

anova(model2,model3, refit = FALSE)


The comparison of model fit between the random-intercept model (model2) and the random-slope model (model3) is assessed through an ANOVA. The significant Chi-square value (Chisq = 25.26, df = 2, p < 0.001) indicates that the random-slope model provides a significantly better fit to the data compared to the random-intercept model. This suggests that the inclusion of random slopes for certain predictors in model3 significantly improves its fit, leading us to prefer the more flexible random-slope model over the more restrictive random-intercept model.

## Random Slope for Two Level-1 Factors

In [ ]:
#
model4 <- lmer(Emotion ~ 1 + Sex +  Imagination + (1 + Sex +  Imagination | Faculty) , data = data)
summary(model4)


Sometimes we may find a warning message saying that the model failed to converge. Linear mixed-effects models can be computationally complex, especially when they have rich random-effects structures, and failure to converge basically means that a good fit for the data could not be found within a reasonable number of iterations of attempting to estimate model parameters. It is important never to report the results of a nonconverging model, as the convergence warnings are an indication that the model has not been reliably estimated and therefore cannot be trusted.

The first step you should take to address convergence issues is to consider our data set and how your model relates to it, and to ensure that our model has not been misspecified (e.g., have we included by-item random slopes for a predictor that does not actually vary within items?). It is also possible that the convergence warnings stem from imbalanced data:. Sometimes we may use some control parameters, depending on the source of the convergence issues, some may be more appropriate or useful than others.

In [ ]:
model4 <- lmer(Emotion ~ 1 + Sex +  Imagination + (1 + Sex +  Imagination | Faculty) ,control = lmerControl(optimizer = "bobyqa"), data = data)
summary(model4)

In [ ]:
plot(model4)

In this linear mixed-effects model:

Random effects for Faculty include random intercepts, slopes for Sex2, and slopes for Imagination.
Fixed effects indicate that, on average, participants with Sex2 have a slightly higher Emotion score (estimate = 0.095, p = 0.026), and each one-unit increase in Imagination is associated with an increase of approximately 0.48 units in the predicted Emotion score (p < 0.001).
The correlations among fixed effects show negligible correlation between the intercept and Sex2 but a strong negative correlation between the intercept and Imagination, indicating that faculties with higher baseline Emotion scores tend to have a smaller positive effect of Imagination.


## Comparison between model4 and modle3

In [ ]:
# compare model 4 and model 3

anova(model4,model3, refit = FALSE)


The comparison of model fit between the random-intercept model (model3) and the random-slope model (model4) is assessed through an ANOVA.
Using an ANOVA reveals that the inclusion of additional predictors in Model 4 does not significantly enhance the fit to the data (Chi-sq = 0.494, df = 4, p = 0.974). Consequently, the simpler Model 3, characterized by a lower AIC and BIC, is deemed more parsimonious and is favored for its comparable explanatory power.

## Coefficients of model 3

In [ ]:
coef(model3)

This output indicates that the estimated intercept for the Faculty1 is 2.93, and the estimated slope is 0.30; these values are almost similar to the estimates for the fixed intercept (2.10) and slope (0.48 ms).

> We can further explore different types and combinations.